In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd
import seaborn as sns; print(sns.__version__)
from datetime import datetime

import pingouin as pg
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))


print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

In [ ]:
from config2 import path_data
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from behav_proc import *

import numpy as np
#from collections import OrderedDict as odict
from config2 import envcode2env
from pingouin import ttest

from base2 import radius_cursor
radius_cursor

In [ ]:
fnf = pjoin(path_data,'df_all_multi_tsz_.pkl.zip')
#fnf = '/home/demitau/ownCloud/Current/merr_data/df_all_multi_tsz.pkl.zip'
print(fnf)
print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
df_all_multi_tsz = pd.read_pickle(fnf)
#dfc_multi_tsz = pd.   read_pickle(pjoin(path_data,'df_all_multi_tsz_trunc_q=0.05.pkl.zip'))
df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                            ' and retention_factor_s == "1.000"').copy()
df_orig = df

assert not df.duplicated(['subject','trials']).any()
#del df_all_multi_tsz
#df['env'] = df['environment'].apply(lambda x: envcode2env[x])
assert 'env' in df
df['feedback_deg'] = df['feedback'] / np.pi * 180
df['error_deg'] = df['error'] / np.pi * 180 

In [ ]:
dfni = df[~np.isinf(df['err_sens'])]
std_mult = 5
ES_thr = dfni_d[dfni_d.columns[1:]].mean().to_dict()['std'] * std_mult
dfni_g = dfni.query('err_sens.abs() <= @ES_thr')
nremoved_pooled = len(dfni) - len(dfni_g)

sz = dfni.groupby(['subject']).size()
sz_g = dfni_g.groupby(['subject']).size()
mpct = ((sz - sz_g) / sz).mean() * 100
print(f'Mean percentage of removed trials = {mpct:.2f}%, '
      f'pooled = {nremoved_pooled / len(dfni) * 100:.2f}%')

In [ ]:
print('Largetst ES')
display( df.loc[df['err_sens'].nlargest(10).index,
      ['subject_ind','trials','err_sens','correction',
       'vals_for_corr1','vals_for_corr2',
       'target_locs','org_feedback',
      'error','prev_error']] )
dfni = df[~np.isinf(df['err_sens'])]
print('Pooled ES')
display(dfni['err_sens'].describe())

print('Mean of ES stats within particpant')
dfni_d = dfni.groupby(['subject'])['err_sens'].describe().reset_index()#.mean(numeric_only=True)
dfni_d[dfni_d.columns[1:]].mean().to_frame().T

In [ ]:
std_mult = 5
ES_thr = dfni_d[dfni_d.columns[1:]].mean().to_dict()['std'] * std_mult
dfni_g = dfni.query('err_sens.abs() <= @ES_thr')
len(dfni_g), len(dfni), len(dfni) - len(dfni_g)

In [ ]:
sz = dfni.groupby(['subject']).size()
sz_g = dfni_g.groupby(['subject']).size()
display('N removed trials',sz - sz_g)
mpct = ((sz - sz_g) / sz).mean() * 100
print(f'Mean percentage of removed trials = {mpct}%')

In [ ]:
dftmp = df[~np.isinf(df['err_sens'])].query('err_sens.abs() > 10')
tr = list(set( (dftmp.trials.values) - 1 ) | set(dftmp.trials.values))
sinds = list(sorted( (dftmp.subject_ind) ) )
print(tr,sinds)
dftmp = df.query('subject_ind.isin(@sinds) and trials.isin(@tr)')
dftmp = dftmp.reset_index()
dftmp[['subject_ind','trials','err_sens','correction',
       'vals_for_corr1','vals_for_corr2',
       'target_locs','org_feedback',
      'error','prev_error']]

In [ ]:
from figure.plots import relplot_multi
dftmp = df.query(
    'subject_ind == 6 '
    ' and trials >= 523 and trials <= 528')
print( len(dftmp) )
r= relplot_multi(data=dftmp,
              ys=[['err_sens'],
                  ['correction'],                  
                  ['vals_for_corr1'],
                  ['movement_duration'],
                  ['home_duration'],
                  ['error'],
                 ['target_locs'],
                 ['org_feedback'],
                 ], x='trials',
                height=1.7,aspect=4)
r[0].refline(y=0)
r[0].refline(x=526)

In [ ]:
from figure.plots import relplot_multi
dftmp = df.query(
    'subject_ind == 4 '
    ' and trials >= 303 and trials <= 308')
print( len(dftmp) )
r= relplot_multi(data=dftmp,
              ys=[['err_sens'],
                  ['correction'],                  
                  ['vals_for_corr1'],
                  ['movement_duration'],
                  ['home_duration'],
                  ['error'],
                 ['target_locs'],
                 ['org_feedback'],
                 ], x='trials',
                height=1.7,aspect=4)
r[0].refline(y=0)
r[0].refline(x=307)
# on trial 306 belief_ is very large, on 307 it is small so 
# corretion is big at 307
# why belief_ is large on 306? because it is =
# target_locs - org_feedback and target_locs is pos and org_feedback is neg

In [ ]:
from figure.plots import relplot_multi
dftmp = df.query(
    'subject_ind == 4  '
    ' and trials >= 0 and trials <= 708')
print( len(dftmp) )
r= relplot_multi(data=dftmp,
              ys=[['err_sens'],
                  ['correction'],                  
                  ['vals_for_corr1'],
                  ['movement_duration'],
                  ['home_duration'],
                  ['error'],
                 ['target_locs'],
                 ['org_feedback'],
                 ], x='trials',
                height=2,aspect=3)
r[0].refline(y=0)
r[0].refline(x=307)

In [ ]:
sns.pairplot(data=dftmp, 
    x_vars=['movement_duration','home_duration',
            'pre_break_duration'],
    y_vars = ['error','err_sens','correction','vals_for_corr1'])